In [160]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [190]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Read df

In [162]:
#this function is to read, transform and join 2 data frame

def read_features():
    path = 'secom.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['feature_'+str(x+1) for x in range(len(df.columns))]
    return df



def read_target():
    path = 'secom_labels.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['status','timestamp']
    df['timestamp'] = pd.to_datetime(df['timestamp'],dayfirst=True)
    return df

#for the testing purporse, trim to remain first 100 rows only
X = read_features()
y = read_target().iloc[:,0]

In [163]:
X.shape

(1567, 590)

# Remove duplicated columns

In [164]:
#find the duplicated features (columns)
def remove_duplicated_columns(df):
    dict_duplicate_pair = {}
    dict_duplicate_matches = {}
    list_duplicate = []
    to_remove = []
    for i in range(0, len(df.columns)):
        l = []
        for j in range(i+1,len(df.columns)):
            dict_duplicate_pair[str(i+1)+';'+str(j+1)] = df.iloc[:,i].equals(df.iloc[:,j])
            if df.iloc[:,i].equals(df.iloc[:,j]) == True:
                if j not in list_duplicate:
                    l.append(j)
                    to_remove.append('feature_'+str(j+1))
                list_duplicate.append(i)
                list_duplicate.append(j)
        if len(l)!=0:
            dict_duplicate_matches[i] = l


    df_duplicate_pair = pd.DataFrame.from_dict(dict_duplicate_pair, orient='index')
    df_duplicate_pair.columns=['duplicate']

    df_duplicate_matches = pd.DataFrame.from_dict(dict_duplicate_matches, orient='index')

    
    df = df.drop(columns=to_remove, axis = 1)

    return df

X = remove_duplicated_columns(X)
X.shape


(1567, 486)

# Remove columns with Constant volatility (std=0)

In [165]:
X.shape

(1567, 486)

In [166]:
def remove_constant_volatility(df):
    df_EDA= df.describe().T
    df_EDA= df_EDA[df_EDA["std"] == 0]
    df = df.drop(axis=1, columns=df_EDA.index)
    return df

X = remove_constant_volatility(X)
X.shape

(1567, 474)

# Remove columns with high %Missing values

In [167]:
X.shape

(1567, 474)

In [168]:
def remove_cols_with_high_pct_null(df, null_threshold):
    list_column_with_pct_null = pd.concat([df.isnull().sum(), df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    list_column_with_pct_null= list_column_with_pct_null[list_column_with_pct_null["Percentage (%)"] >= null_threshold]
    df = df.drop(axis=1, columns=list_column_with_pct_null.index)
    return df

X = remove_cols_with_high_pct_null(X, 0.8)
X.shape

(1567, 466)

# Split data

In [169]:
X

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_7,feature_8,feature_9,feature_10,feature_11,...,feature_581,feature_582,feature_583,feature_584,feature_585,feature_586,feature_587,feature_588,feature_589,feature_590
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,97.6133,0.1242,1.5005,0.0162,-0.0034,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.4966,-0.0005,-0.0148,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,95.4878,0.1241,1.4436,0.0041,0.0013,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,104.2367,0.1217,1.4882,-0.0124,-0.0033,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5031,-0.0031,-0.0072,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2899.41,2464.36,2179.7333,3085.3781,1.4843,82.2467,0.1248,1.3424,-0.0045,-0.0057,...,0.0047,203.1720,0.4988,0.0143,0.0039,2.8669,0.0068,0.0138,0.0047,203.1720
1563,3052.31,2522.55,2198.5667,1124.6595,0.8763,98.4689,0.1205,1.4333,-0.0061,-0.0093,...,NaN,NaN,0.4975,0.0131,0.0036,2.6238,0.0068,0.0138,0.0047,203.1720
1564,2978.81,2379.78,2206.3000,1110.4967,0.8236,99.4122,0.1208,NaN,NaN,NaN,...,0.0025,43.5231,0.4987,0.0153,0.0041,3.0590,0.0197,0.0086,0.0025,43.5231
1565,2894.92,2532.01,2177.0333,1183.7287,1.5726,98.7978,0.1213,1.4622,-0.0072,0.0032,...,0.0075,93.4941,0.5004,0.0178,0.0038,3.5662,0.0262,0.0245,0.0075,93.4941


In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1,stratify=y)

# Missing value & Outlier treatment

In [171]:
#Outlier treatment
def outlierknn(df):
    #Outlier treatment first:
    df =df.mask(df.sub(df.mean()).div(df.std()).abs().gt(3))
    #Scaling data 
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns= df.columns)
    #KNN Imputation 
    knn = KNNImputer(n_neighbors=20, weights='uniform')
    df = pd.DataFrame(knn.fit_transform(df), columns=df.columns)
    #Reverse Scaling
    df = pd.DataFrame(scaler.inverse_transform(df), columns= df.columns)
    return df

X_train = outlierknn(X_train)

# Feature Selection

In [172]:
X_train.shape

(1253, 466)

In [173]:
#Boruta function with random forest

def FeatureSelection_Boruta(X, y, bor_iter) :
    feature_names = np.array(X.columns)

    # define random forest classifier
    model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
    model.fit(X, y)
    
    # define Boruta feature selection method
    feature_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=100, max_iter=bor_iter)

    # find all relevant features
    feature_selector.fit(X.to_numpy(),y)

    # check selected features
    feature_selector.support_

    # check ranking of features
    feature_ranking=feature_selector.ranking_

    # zip feature names, ranks, and decisions 
    feature_ranks = list(zip(feature_names, 
                             feature_selector.ranking_, 
                             feature_selector.support_))

    # print the results
    for feat in feature_ranks:
        print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features = list()
    indexes = np.where(feature_selector.ranking_ <= 1)
    for x in np.nditer(indexes):
        final_features.append(feature_names[x])
    print(final_features)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features))


X_train = FeatureSelection_Boruta(X_train, y_train, 140)

X_train.shape

Iteration: 	1 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	2 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	3 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	4 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	5 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	6 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	7 / 140
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	8 / 140
Confirmed: 	0
Tentative: 	14
Rejected: 	452
Iteration: 	9 / 140
Confirmed: 	1
Tentative: 	13
Rejected: 	452
Iteration: 	10 / 140
Confirmed: 	1
Tentative: 	13
Rejected: 	452
Iteration: 	11 / 140
Confirmed: 	1
Tentative: 	13
Rejected: 	452
Iteration: 	12 / 140
Confirmed: 	3
Tentative: 	11
Rejected: 	452
Iteration: 	13 / 140
Confirmed: 	3
Tentative: 	11
Rejected: 	452
Iteration: 	14 / 140
Confirmed: 	3
Tentative: 	11
Rejected: 	452
Iteration: 	15 / 140
Confirmed: 	3
Tentative: 	11
Rejected: 	452
Iteration: 	16 / 140
Confirmed: 	3
Tentat

(1253, 12)

In [174]:
X_train.to_csv('output/X_train')
y_train.to_csv('output/y_train')

# Balancing

In [156]:
# X_train = pd.read_csv('output/X_train').iloc[:,1:]
# y_train = pd.read_csv('output/y_train').iloc[:,1:]

In [158]:
X_train.shape

(1253, 12)

In [175]:
def Sampling(X_train, y_train, sampler):
    
    #SMOTE
    if sampler == 'SMOTE':
        sampler = SMOTE(random_state=100)    
    
    #ROSE
    if sampler == 'ROSE':
        sampler = RandomOverSampler(random_state=100, shrinkage=1)

    #ADASYN
    if sampler == 'ADASYN':
        sampler = ADASYN(random_state=100)
    

    #SMOTTEENN
    if sampler == 'SMOTEENN' :
        sampler = SMOTEENN(random_state=100)
        
        
    #Random under Sampling
    if sampler == "randomunder":
        sampler = RandomUnderSampler(random_state=100)

    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    counter = Counter(y_resampled)
    print(counter)
    return X_resampled, y_resampled

X_train, y_train = Sampling(X_train, y_train,'SMOTE')
X_train.shape

Counter({-1: 1170, 1: 1170})


(2340, 12)

# Transform Test split

In [186]:
#Remove cols that do not exist in Train data
X_test = X_test.loc[:,X_train.columns]

In [187]:
X_test = outlierknn(X_test)

# Model

In [188]:
X_test.shape

(314, 12)

In [199]:
def run_model(X_train, y_train, X_test, y_test):
    # building model before balancing data
    model = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
    model.fit(X_train,y_train)
    y_pred= model.predict(X_test)
    score1= accuracy_score(y_test, y_pred)
    cf_matrix = confusion_matrix(y_test, y_pred)
    specificity = cf_matrix[1][1] / ( cf_matrix[1][1] + cf_matrix[1][0] )
    return score1, specificity, cf_matrix

run_model(X_train, y_train, X_test, y_test)

In [ ]:
list_null_impute = [knnimputation_distance, MICEimputation_distance]
list_null_outlier = [outlier_knn,outlier_3s]
list_feat_selection = ['Boruta_RF', 'Boruta_shap', 'RFE']
Boruta = ['RF', 'XGB']
Boruta_shap = ['RF', 'XGB', 'kNN']
RFE = ['RF', 'SVC']

In [ ]:
remove_duplicated_columns()
remove_constant_volatility()
...
for i in list_null_impute:
    for o in list_null_outlier:
        for fs in ...:
            run_model(i,o,fs)
